In [10]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
import numpy as np
import os
import copy
import matplotlib.pyplot as plt
import sys, time, os, random
from os.path import join as oj
from tqdm import tqdm
from skimage.transform import resize

import visualize as viz
import tiling
import agglomerate
import operator
import pickle
from cd import cd

import torch
import torch.nn.functional as F
from torch.autograd import Variable
import imagenet.imagenet as dset
import scores as score_funcs

from agglomerate import get_scores
import torchvision.datasets as datasets
from torchvision import models

# get model
# model = models.alexnet(pretrained=True).cuda().eval()
model = models.vgg16(pretrained=True).cuda().eval()
model_type='vgg' # alexnet, vgg

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [11]:
method = 'cd'
sweep_dim = 14
layer = 'softmax'
use_abs = False
num_iters = 8
percentile_include = 96
batch = False
_, val_loader = dset.load_data()

# find best, worst examples

In [4]:
preds = []
print(len(val_loader))
for i, im in enumerate(val_loader):
    im_torch, label = Variable(im[0].cuda()), im[1].numpy()[0]
    if i % 1000 == 0:
        print(i)
    pred = model(im_torch).data[0, label]
    preds.append(pred)
pickle.dump(preds, open('preds_imagenet.pkl', 'wb'))

50000
0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000


In [7]:
preds = pickle.load(open('preds_imagenet.pkl', 'rb'))
top_idxs = np.argsort(preds)[::-1]
preds[top_idxs[0]]
top_10 = top_idxs[:10]
bot_10 = top_idxs[-10:][::-1]
print('top idxs', top_10)
print('bot idxs', bot_10)

top idxs [30246 30209 27731 21483  1472 30217 30247 30596 20897  7065]
bot idxs [46516 48180 15466 34520 45305 23247 19298 28353 12554  8366]


In [12]:
# Generate bad model
np.random.seed(17042018)
modify_prop = 0.008
bad_model = copy.deepcopy(model).eval()
for param in bad_model.parameters():
    num_params = np.prod(list(param.data.size()))
    num_modify = modify_prop * num_params
    to_modify = np.random.choice([True, False], tuple(param.data.size()), 
                                 p=(modify_prop, 1-modify_prop))
    cpu_param = param.data.cpu().numpy()
    #permuted_vars = cpu_param[to_modify]
    cpu_param[to_modify] = np.random.permutation(cpu_param[to_modify])
    param.data = torch.cuda.FloatTensor(cpu_param) 


In [14]:
good_exs = [0] #4195, 22182, 35727, 1939, 4020, 6910]
bad_exs = [] #[7224, 28123, 48736]
all_data = {}
for i, im in enumerate(val_loader):
    if i % 100 == 0:
        print(i, end=' ')
    if i in good_exs or i in bad_exs:
        all_data[i] = im
    if i > max(good_exs):
        break
    

#all_data = {i:im for i, im in enumerate(val_loader) if i in ex_list}

0 

In [4]:
def predict(model, im):
    pred = model(im)
    _, pred = pred[0].max(0)
    return pred.data[0]

In [21]:
# Compute accuracy of good, bad model
num_correct, num_good_correct, num_total = 0, 0, 0

for i, im in enumerate(val_loader):
    im_torch, im_orig, label = Variable(im[0].cuda()), im[0].numpy().squeeze(), im[1].numpy()[0]
    pred = bad_model(im_torch)
    _, pred = pred[0].max(0)
    pred = pred.data[0]
    if pred == label:
        num_correct += 1
        
    pred = model(im_torch)
    _, pred = pred[0].max(0)
    pred = pred.data[0]
    if pred == label:
        num_good_correct += 1
        
    num_total += 1
    if num_total % 1000 == 0:
        print(num_total)
    #if num_total > 10000:
        #break

print("Accuracy of bad model: ", num_correct / num_total)
print("Accuracy of good model: ", num_good_correct / num_total)

1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000
Accuracy of bad model:  0.32374
Accuracy of good model:  0.42778


In [17]:
F.softmax(bad_model(im_torch)).data[0][good_pred]

/usr/local/linux/anaconda3/lib/python3.5/site-packages/ipykernel_launcher.py:1: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  """Entry point for launching an IPython kernel.


0.07658673077821732

In [47]:
good_int_sum = [[] for i in range(len(val_loader))]
bad_int_sum = [[] for i in range(len(val_loader))]
good_int_sums = [[] for i in range(len(val_loader))]
bad_int_sums = [[] for i in range(len(val_loader))]
decomp_cnt = 0
ex_list = set([])
good_correct = {}
bad_correct = {}
for i, im in enumerate(val_loader):
    im_torch, im_orig, lab_num = Variable(im[0].cuda()), im[0].numpy().squeeze(), im[1].numpy()[0]
    good_max, good_pred = F.softmax(model(im_torch)).max(1)
    good_pred = good_pred.data[0]; good_max = good_max.data[0]
    bad_max, bad_pred = F.softmax(bad_model(im_torch)).max(1)
    bad_pred = bad_pred.data[0]; bad_max = bad_max.data[0]
    
    if (good_pred == lab_num or bad_pred == lab_num) and good_pred != bad_pred:
        if good_pred == lab_num:
            good_correct[i] = good_max - F.softmax(bad_model(im_torch)).data[0, good_pred]
        else:
            bad_correct[i] = bad_max - F.softmax(model(im_torch)).data[0, bad_pred]
            
    if False and good_pred != bad_pred and (good_pred == lab_num or bad_pred == lab_num) and good_max > 0.5 and bad_max > 0.5:
        ex_list |= set([i])
        
        if False:
            #lab_pred = np.argmax(dset.pred_ims(model, np.copy(im_orig)))

            lists = agglomerate.agglomerate(model, dset.pred_ims, percentile_include, 
                                            method, sweep_dim, layer, im_orig, 
                                            good_pred, use_abs, num_iters=num_iters, 
                                            im_torch=im_torch, batch=False, model_type='vgg')
            int_sum, int_sums = agglomerate.sum_interactions(lists)
            good_int_sum[i] = int_sum
            good_int_sums[i] = int_sums
            lists = agglomerate.agglomerate(bad_model, dset.pred_ims, percentile_include, 
                                            method, sweep_dim, layer, im_orig, 
                                            bad_pred, use_abs, num_iters=num_iters, 
                                            im_torch=im_torch, model_type='vgg')
            int_sum, int_sums = agglomerate.sum_interactions(lists)
            bad_int_sum[i] = int_sum
            bad_int_sums[i] = int_sums
        decomp_cnt += 1
    if i % 2000 == 0:
        print(i, decomp_cnt, len(ex_list))
      
    

/usr/local/linux/anaconda3/lib/python3.5/site-packages/ipykernel_launcher.py:11: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  # This is added back by InteractiveShellApp.init_path()
/usr/local/linux/anaconda3/lib/python3.5/site-packages/ipykernel_launcher.py:13: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  del sys.path[0]
/usr/local/linux/anaconda3/lib/python3.5/site-packages/ipykernel_launcher.py:18: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
/usr/local/linux/anaconda3/lib/python3.5/site-packages/ipykernel_launcher.py:20: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


0 0 0
2000 0 0
4000 0 0
6000 0 0
8000 0 0
10000 0 0
12000 0 0
14000 0 0
16000 0 0
18000 0 0
20000 0 0
22000 0 0
24000 0 0
26000 0 0
28000 0 0
30000 0 0
32000 0 0
34000 0 0
36000 0 0
38000 0 0
40000 0 0
42000 0 0
44000 0 0
46000 0 0
48000 0 0


In [48]:
sorted_good = sorted(good_correct.items(), key=operator.itemgetter(1))[::-1]
sorted_bad = sorted(bad_correct.items(), key=operator.itemgetter(1))[::-1]
print(sorted_bad[:5])
print(sorted_good[:5])

[(28123, 0.8197124898433685), (7224, 0.8181415572762489), (48736, 0.6984506845474243), (2446, 0.6785046607255936), (1479, 0.6623021401464939)]
[(4020, 0.9969721467932686), (22182, 0.9896679029334337), (1939, 0.987184027209878), (6910, 0.9856045837514102), (35727, 0.9844401758164167)]


In [41]:
im_torch, im_orig, lab_num = dset.get_im_and_label(1469)

In [46]:
good_max, good_pred = F.softmax(model(im_torch)).max(1)
print(good_max, good_pred)
F.softmax(bad_model(im_torch)).data[0, good_pred.data[0]]

Variable containing:
 0.7241
[torch.cuda.FloatTensor of size 1 (GPU 0)]
 Variable containing:
 29
[torch.cuda.LongTensor of size 1 (GPU 0)]



/usr/local/linux/anaconda3/lib/python3.5/site-packages/ipykernel_launcher.py:1: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  """Entry point for launching an IPython kernel.
/usr/local/linux/anaconda3/lib/python3.5/site-packages/ipykernel_launcher.py:3: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  This is separate from the ipykernel package so we can avoid doing imports until


0.8768461346626282

In [49]:
good_exs = set([a[0] for a in sorted_good[:6]])
bad_exs = set([a[0] for a in sorted_bad[:3]])

In [6]:
with open('data/out_imagenet_good_out.pkl', 'wb') as fout:
    pickle.dump({'int_sums': good_int_sums, 'int_sum': good_int_sum}, fout)
    
with open('data/out_imagenet_bad_out.pkl', 'wb') as fout:
    pickle.dump({'int_sums': bad_int_sums, 'int_sum': bad_int_sum}, fout)
    
    

NameError: name 'pickle' is not defined

In [50]:
with open('data/out_imagenet_good_correct.pkl', 'wb') as fout:
    pickle.dump(good_correct, fout)
    
with open('data/out_imagenet_bad_correct.pkl', 'wb') as fout:
    pickle.dump(bad_correct, fout)
    

In [38]:
with open('data/out_imagenet_good_correct.pkl', 'rb') as fin:
    good_correct = pickle.load(fin)
    
with open('data/out_imagenet_bad_correct.pkl', 'rb') as fin:
    bad_correct = pickle.load(fin)
    

In [63]:
bad_model(im_torch)

Variable containing:
  3.1475   4.4626  11.6072  ...    2.8309   1.6987   1.2151
[torch.cuda.FloatTensor of size 1x1000 (GPU 0)]

In [5]:
sweep_dim = 14
for i in good_exs:
    print(i)
    for mod in [model, bad_model]:
        im = all_data[i]
        im_torch, im_orig, lab_num_correct = Variable(im[0].cuda()), np.copy(im[0].numpy()).squeeze().transpose((1, 2, 0)), im[1].numpy()[0]

        lab_pred = np.argmax(dset.pred_ims(mod, np.copy(im_orig)))
        pred = mod(im_torch)
        _, pred = pred[0].max(0)
        pred = pred.data[0]
        lists = agglomerate.agglomerate(mod, dset.pred_ims, percentile_include, 
                                        method, sweep_dim, layer, im_orig, 
                                        pred, use_abs, num_iters=num_iters, 
                                        im_torch=im_torch, batch=batch, model_type=model_type)        
        im_orig.transpose((1, 2, 0))
        # visualize
        plt.figure(figsize=(12, 11), facecolor='white')

        rows = 7       
        num_ims = len(lists['scores_list'])

        # original plots
        ind, labs = viz.visualize_original_preds(im_orig, lab_num_correct, 
                                                 lists['comp_scores_raw_list'], lists['scores_orig_raw'],
                                                 subplot_rows=rows, dset=dset)

        # comp plots
        viz.visualize_ims_list(lists['scores_list'], 
                               cmap_new='redwhiteblue',
                               title=dset.lab_dict[lab_pred] + ' scores',
                               subplot_row=2, subplot_rows=rows, colorbar=True)
        viz.visualize_ims_list(lists['comps_list'],
                              title='Chosen blobs',
                              subplot_row=3, subplot_rows=rows, colorbar=False, im_orig=im_orig, plot_overlay=True)

        # dict plots
        viz.visualize_dict_list_top(lists['comp_scores_raw_list'], method,
                               subplot_row=4, subplot_rows=rows, ind=ind, labs=labs, use_orig_top=True)
        viz.visualize_dict_list_top(lists['comp_scores_raw_list'], method,
                        subplot_row=6, subplot_rows=rows, ind=ind, 
                        dset=dset, use_orig_top=False)
        plt.show()
    #print(ex_ind)
    #if ex_ind > 10:
        #break


0


KeyboardInterrupt: 

In [6]:
sweep_dim = 14
for i in bad_exs:
    print(i)
    for mod in [model, bad_model]:
        im = all_data[i]
        im_torch, im_orig, lab_num_correct = Variable(im[0].cuda()), np.copy(im[0].numpy()).squeeze().transpose((1, 2, 0)), im[1].numpy()[0]

        lab_pred = np.argmax(dset.pred_ims(mod, np.copy(im_orig)))
        pred = mod(im_torch)
        print(pred)
        _, pred = pred[0].max(0)
        pred = pred.data[0]
        lists = agglomerate.agglomerate(mod, dset.pred_ims, percentile_include, 
                                        method, sweep_dim, layer, im_orig, 
                                        pred, use_abs, num_iters=num_iters, 
                                        im_torch=im_torch, batch=batch, model_type=model_type)        
        im_orig.transpose((1, 2, 0))
        # visualize
        plt.figure(figsize=(12, 11), facecolor='white')

        rows = 7       
        num_ims = len(lists['scores_list'])

        # original plots
        ind, labs = viz.visualize_original_preds(im_orig, lab_num_correct, 
                                                 lists['comp_scores_raw_list'], lists['scores_orig_raw'],
                                                 subplot_rows=rows, dset=dset)

        # comp plots
        viz.visualize_ims_list(lists['scores_list'], 
                               cmap_new='redwhiteblue',
                               title=dset.lab_dict[lab_pred] + ' scores',
                               subplot_row=2, subplot_rows=rows, colorbar=True)
        viz.visualize_ims_list(lists['comps_list'],
                              title='Chosen blobs',
                              subplot_row=3, subplot_rows=rows, colorbar=False, im_orig=im_orig, plot_overlay=True)

        # dict plots
        viz.visualize_dict_list_top(lists['comp_scores_raw_list'], method,
                               subplot_row=4, subplot_rows=rows, ind=ind, labs=labs, use_orig_top=True)
        viz.visualize_dict_list_top(lists['comp_scores_raw_list'], method,
                        subplot_row=6, subplot_rows=rows, ind=ind, 
                        dset=dset, use_orig_top=False)
        plt.show()
    #print(ex_ind)
    #if ex_ind > 10:
        #break


# generate mturk viz examples

In [ ]:
sweep_dim = 14
rows = 2
out_dir = 'vision_results/imagenet'
for i, m in enumerate([model, bad_model]):
    model_name = ['model=good', 'model=bad'][i]
    for ex in good_exs + bad_exs:
        ex_fold = 'ex=good' if ex in good_exs else 'ex=bad'
        
        
        # get data[ex]
        im = all_data[ex]
        im_torch, im_orig, lab_num_correct = Variable(im[0].cuda()), np.copy(im[0].numpy()).squeeze().transpose((1, 2, 0)), im[1].numpy()[0]

        lab_pred = np.argmax(dset.pred_ims(m, np.copy(im_orig)))
        pred = m(im_torch)
        _, pred = pred[0].max(0)
        pred = pred.data[0]
        lists = agglomerate.agglomerate(m, dset.pred_ims, percentile_include, 
                                        method, sweep_dim, layer, im_orig, 
                                        pred, use_abs, num_iters=num_iters, 
                                        im_torch=im_torch, batch=batch, model_type=model_type)
        

        # ACD
        fig_name = str(ex) + model_name + '_method=acd_' + ex_fold
        plt.figure(figsize=(12, 4), facecolor='white')
        ind, labs = viz.visualize_original_preds(im_orig, lab_num_correct, 
                                         lists['comp_scores_raw_list'], lists['scores_orig_raw'],
                                         subplot_rows=rows, dset=dset, mturk=True)
        viz.visualize_ims_list(np.copy(lists['comps_list']), title='', subplot_row=1, subplot_rows=rows, 
                               colorbar=False, im_orig=im_orig, plot_overlay=True, 
                               mturk=True, num_ims=lists['num_before_final'], comp_scores_raw=lists['comp_scores_raw_list'], lab_num_correct=lab_pred) # mturk stuff
        plt.savefig(oj(out_dir, fig_name + '.png'), bbox_inches='tight')

        # CD
        fig_name = str(ex) + model_name + '_method=cd_'+ ex_fold
        plt.figure(figsize=(16, 3), facecolor='white')
        ind, labs = viz.visualize_original_preds(im_orig, lab_num_correct, 
                                         lists['comp_scores_raw_list'], lists['scores_orig_raw'],
                                         subplot_rows=1, dset=dset, mturk=True)
        plt.savefig(oj(out_dir, fig_name + '.png'), bbox_inches='tight')

        # integrated gradients
        fig_name = str(ex) + model_name + '_method=ig_'+ ex_fold
        scores_ig = score_funcs.ig_scores_cnn(model, im_torch, sweep_dim=sweep_dim, num_classes=1000, im_size=224, ind=ind)
        plt.figure(figsize=(16, 3), facecolor='white')
        ind, labs = viz.visualize_original_preds(im_orig, lab_num_correct, 
                                             lists['comp_scores_raw_list'], scores_ig,
                                             subplot_rows=1, dset=dset, mturk=True)
        
        plt.savefig(oj(out_dir, fig_name + '.png'), bbox_inches='tight') 
        
        # break_down
        fig_name = str(ex) + model_name + '_method=break_down_'+ ex_fold
        tiles_break = tiling.gen_tiles(im_orig, fill=0, method='break_down', sweep_dim=sweep_dim)
        preds_break = agglomerate.get_scores(model, method='break_down', ims=tiles_break, 
                                    im_torch=im_torch, pred_ims=dset.pred_ims, layer='softmax')
        plt.figure(figsize=(16, 3), facecolor='white')
        ind, labs = viz.visualize_original_preds(im_orig, lab_num_correct, 
                                             lists['comp_scores_raw_list'], preds_break,
                                             subplot_rows=1, dset=dset, mturk=True)
        plt.savefig(oj(out_dir, fig_name + '.png'), bbox_inches='tight')


/accounts/projects/vision/chandan/agglomerative_attention/agglomerate.py:83: RuntimeWarning: invalid value encountered in greater_equal
  im_thresh = np.logical_and(scores >= thresh, ~np.isnan(scores))


# generate supp viz

In [20]:
good_exs = [4195, 22182, 35727, 1939, 4020, 6910]
bad_exs = [7224, 28123, 48736]
all_data = pickle.load(open("imagenet_examples.pkl", "rb"))
good_idxs = list(all_data.keys())[0]
sweep_dim = 56
rows = 4
out_dir = 'vision_results/imagenet/supp'
if not os.path.exists(out_dir):
    os.makedirs(out_dir)
for i, m in enumerate([model, bad_model]):
    model_name = ['model=good', 'model=bad'][i]
    for ex in good_exs + bad_exs:
        ex_fold = 'ex=good' if ex in good_exs else 'ex=bad'
        
        
        # get data[ex]
        im = all_data[ex]
        im_torch, im_orig, lab_num_correct = Variable(im[0].cuda()), np.copy(im[0].numpy()).squeeze().transpose((1, 2, 0)), im[1].numpy()[0]

        lab_pred = np.argmax(dset.pred_ims(m, np.copy(im_orig)))
        pred = m(im_torch)
        _, pred = pred[0].max(0)
        pred = pred.data[0]
        lists = agglomerate.agglomerate(m, dset.pred_ims, percentile_include, 
                                        method, sweep_dim, layer, im_orig, 
                                        pred, use_abs, num_iters=num_iters, 
                                        im_torch=im_torch, batch=batch, model_type=model_type)
        

        fig_name = str(ex) + model_name + '_method=acd_' + ex_fold
        
        plt.figure(figsize=(14, 6), facecolor='white', dpi=300)
        ind, labs = viz.visualize_original_preds(im_orig, lab_num_correct, 
                                 lists['comp_scores_raw_list'], lists['scores_orig_raw'],
                                 subplot_rows=rows, dset=dset, mturk=True)
        viz.visualize_ims_list(lists['scores_list'], 
                               cmap_new='redwhiteblue',
                               title='Refined scores',
                               subplot_row=1, subplot_rows=rows, colorbar=True)
        viz.visualize_ims_list(lists['comps_list'],
                              title='',
                              subplot_row=2, subplot_rows=rows, colorbar=False, im_orig=im_orig, plot_overlay=True, num_ims=lists['num_before_final'], comp_scores_raw=lists['comp_scores_raw_list'], lab_num_correct=lab_pred)
        viz.visualize_dict_list(lists['comp_scores_raw_list'], method,
                               subplot_row=3, subplot_rows=rows)
        plt.savefig(oj(out_dir, fig_name + '.png'), bbox_inches='tight')

SyntaxError: invalid syntax (<ipython-input-20-43070e9579f9>, line 45)

In [8]:
# load top_10, bot_10 above
all_data = {}
print(len(val_loader))
for i, im in enumerate(val_loader):
    if i % 1000 == 0:
        print(i, end=' ')
    if i in top_10 or i in bot_10:
        all_data[i] = im

50000
0 1000 2000 3000 4000 5000 6000 7000 8000 9000 10000 11000 12000 13000 14000 15000 16000 17000 18000 19000 20000 21000 

Process Process-2:
Traceback (most recent call last):
  File "/usr/local/linux/anaconda3/lib/python3.5/multiprocessing/process.py", line 252, in _bootstrap
    self.run()
  File "/usr/local/linux/anaconda3/lib/python3.5/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/accounts/projects/vision/.local/lib/python3.5/site-packages/torch/utils/data/dataloader.py", line 42, in _worker_loop
    samples = collate_fn([dataset[i] for i in batch_indices])
  File "/accounts/projects/vision/.local/lib/python3.5/site-packages/torch/utils/data/dataloader.py", line 42, in <listcomp>
    samples = collate_fn([dataset[i] for i in batch_indices])
  File "/usr/local/linux/anaconda3/lib/python3.5/site-packages/torchvision-0.2.0-py3.5.egg/torchvision/datasets/folder.py", line 124, in __getitem__
    img = self.transform(img)
  File "/usr/local/linux/anaconda3/lib/python3.5/site-packages/torchvision-0.2.0-py3.5.egg/torchvision/transforms/transforms.py", line

KeyboardInterrupt: 

In [ ]:
# use top_10
# use bot_10
if not os.path.exists('out'):
    os.makedirs('out')

batches = sent_util.get_batches(range(len(dev_iterator) + 1), train_iterator, dev_iterator, dset='dev') # has batch.text, batch.label
print(top_10, bot_10)
for ex in np.concatenate((top_10, bot_10)):
    ex_fold = 'ex=best' if ex in top_10 else 'ex=worst'
    batch = batches[ex]

    # ACD
    fig_name = str(ex) + '_method=acd_' + ex_fold
    viz_args = agglomerate_and_save(method, batch, subtract, percentile_include, num_iters, 
                      sweep_dim, model, dset, absolute, fig_name=fig_name, use_bad=False, mturk=True)